# NER et Analyse de sentiments

Dans ce cahier de notes, nous explorerons comment détecter les différents sentiments présents dans chaque avis. Le processus sera divisé en deux étapes principales :

- Détection des différentes entités dans un hôtel (restaurant, personnel, etc.): tout ce que le client pourrait décrire.
- Déterminer le sentiment envers l'entité détectée.

# Partie NER

Le NER (Named Entity Recognition) est un domaine de l'analyse du texte qui consiste à détecter et à extraire des entités nommées dans du texte. Les entités nommées sont des mots ou des phrases qui représentent des concepts précis dans le monde réel, tels que les noms de personnes, de lieux, d'organisations, de produits, etc.

Le NER peut être utilisé pour diverses applications, telles que l'analyse de sentiments, la reconnaissance de la parole, la traduction automatique, la recherche d'informations, la classification de documents, etc.

Il existe plusieurs techniques et outils pour effectuer le NER, notamment des modèles statistiques, des réseaux de neurones et des approches basées sur les règles. L'exactitude du NER dépend de la qualité des données d'apprentissage et de la complexité du modèle utilisé.

In [1]:
import pandas as pd
reviews = pd.read_excel('../data/reviews_google.xlsx')

example of a review

In [2]:
reviews.review.iloc[0]

'this hotel has the smallest room, and we have been to other ibis brand hotels across the globe. check in and out were efficient. staffs were average. breakfast was the same during our 5-day stay. room was regularly cleaned and towels were replaced. location was outstanding! there were lots of restaurants nearby, as well as stores. metro was helpful in getting around paris. we will stay again here soon!'

## Détection des entités 

Dans cette partie on s'intèresse à détecter les entités présentes dans un commentaire à travers la structure syntaxique de la phrase. On essaye dans un premier temps de détecter les differents sujets existant dans un avis et après nous explorons tous les différents noms

In [3]:
import spacy
# # Initialisation du modèle de traitement du langage naturel
nlp = spacy.load("en_core_web_sm")

def detect_subjects(review):
    """
    Cette fonction a pour objectif de détecter les sujets d'un avis. Pour ce faire,
    elle utilise un modèle de traitement du langage naturel pour analyser la structure
    syntaxique de l'avis et extraire les mots qui sont des sujets de phrases. Elle exclut également
    les mots qui sont des stop words (mots courants qui n'ont pas de signification spécifique. 
    Enfin, elle renvoie la liste des sujets détectés.
    
    :param review: L'avis à analyser, sous forme de chaîne de caractères.
    :type review: str
    :return: La liste des noms détectés dans l'avis.
    """
    #Analyse de l'avis à l'aide du modèle de traitement du langage naturel
    doc = nlp(review)
    # on définit les stop words
    stopping_words = ["a", "an", "and", "are", "as", "at", "be", "by", "for", "from", "has", "he", "in",
                      "is", "it", "its", "of", "on", "that", "the", "to", "was", "were", "will", "with"]
    subjects = []
    # Pour chaque phrase de l'avis
    for sent in doc.sents:
        # Filtrage des tokens (mots) pour exclure les stop words
        filtered_tokens = [token for token in sent if not token.is_stop]
        # Pour chaque token restant
        for token in filtered_tokens:
            # Si le token est un sujet de phrase
            if token.dep_ == "nsubj":
                # Ajout du token à la liste des sujets
                subjects.append(token.text)
    return subjects



In [4]:
def detect_nouns(review):
    """
    Cette fonction prend en entrée un avis sous forme de chaîne de caractères et renvoie une liste de noms détectés dans un avis.
    
    Pour détecter les noms, elle utilise un modèle de traitement du langage naturel pour analyser chaque mot de l'avis
    et vérifier s'il s'agit d'un nom. Elle exclut également les mots qui sont des stop words.
    
    :param review: L'avis à analyser, sous forme de chaîne de caractères.
    :type review: str
    :return: La liste des noms détectés dans l'avis.
    """
    # Analyse de l'avis à l'aide du modèle de traitement du langage naturel

    doc = nlp(review)
    # Définition des stop words
    stopping_words = ["a", "an", "and", "are", "as", "at", "be", "by", "for", "from", "has", "he", "in",
                      "is", "it", "its", "of", "on", "that", "the", "to", "was", "were", "will", "with"]
    subjects = []
    # Pour chaque mot de l'avis
    for token in doc:
        # Si le mot n'est pas un stop word et qu'il s'agit d'un nom
        if not token.is_stop and token.pos_== "NOUN":
            # Ajout du mot à la liste des noms
            subjects.append(token.text)
    # Renvoi de la liste des noms
    return subjects

On applique les fonctions a notre base de données.

In [5]:
df=reviews.copy()
df["subjects"] = df["review"].apply(detect_subjects)
df["subjects"]=df["subjects"].apply(lambda x: [s for s in x if s not in["hotel","value"] ])

In [6]:
df["nouns"] = df["review"].apply(detect_nouns)
#df["subjects"]=df["subjects"].apply(lambda x: [s for s in x if s not in["hotel","value"] ])

exemple:

In [7]:
i=177
rv=df.review.iloc[i]
sbj=detect_subjects(rv)
sbj2=detect_nouns(rv)
#determine_sentiment(rv, sbj)
print(rv)
print(sbj)
print(sbj2)

fantastic position, close to the metro, from where all places are attainable. buy a five day metro ticket, saves the hassle of paying each time. clean rooms, helpful staff, modern facilities, wifi, breakfast is about €10 a-day. meats, cheese, cold breakfast as you say. no kettles in the rooms, for those who are not from europe. i would easy stay here again, location is everything, and a good price. valve for money, it has to be 10/10.
['places', 'rooms', 'breakfast', 'location']
['position', 'places', 'day', 'ticket', 'hassle', 'time', 'rooms', 'staff', 'facilities', 'wifi', 'breakfast', 'day', 'meats', 'cheese', 'breakfast', 'kettles', 'rooms', 'location', 'price', 'money']


In [8]:
df

,index,title,review,rate,trip_type,date,subjects,nouns
0,0,small room but okey,"this hotel has the smallest room, and we have ...",4,traveled as a couple,2022-11-01,"[check, staffs, breakfast, location, metro]","[hotel, room, brand, hotels, globe, staffs, br..."
1,1,"great location, great staff",great location and simple but clean ammenities...,5,traveled on business,2022-10-01,[staff],"[location, ammenities, value, money, stars, st..."
2,2,dont stay in this hotel unless you would like ...,don't stay in this hotel unless you would like...,1,NaN,2022-10-01,"[reservation, staff, room, staff]","[hotel, cancellation, email, day, check, room,..."
3,3,waste of money,not worth it. better rooms available for this ...,1,NaN,2022-09-01,"[rooms, location, 20min, staff]","[rooms, price, rooms, bathrooms, location, wal..."
4,4,pre-autorisation money has not been refunded,ibis paris tour eiffel cambronne charged my de...,3,traveled as a couple,2022-10-01,"[cambronne, money]","[eiffel, cambronne, debit, account, eur, pre, ..."
...,...,...,...,...,...,...,...,...
8564,907,just papering over the cracks!,just got back from 3 nights in paris\none nigh...,3,traveled on business,2010-01-01,"[location, areas, lounge, rooms, staff]","[nights, night, star, nights, hotel, location,..."
8565,908,perfect,our stay was perfect. the location was amazing...,5,traveled as a couple,2008-08-01,"[stay, location, room]","[stay, location, stones, room, view]"
8566,909,very nice hotel,we decided on this hotel in paris for location...,4,traveled with family,2009-05-01,"[employees, service, bathrooms, fi, signal, br...","[hotel, location, view, count, location, rate,..."
8567,910,ideally situated,very small hotel but ideally situated with bre...,4,traveled as a couple,2008-07-01,"[rooms, hallways, room, staff, breakfast, croi...","[hotel, views, river, room, boutique, hotels, ..."


"Les deux fonctions detect_subjects et detect_nouns ont pour objectif d'extraire des informations utiles d'un avis de voyage, respectivement les sujets et les noms mentionnés dans l'avis.

La fonction **detect_subjects** a l'avantage de ne renvoyer que les sujets de phrases, ce qui permet de cibler précisément les éléments sur lesquels l'avis porte. Cependant, elle peut manquer certains noms importants qui ne sont pas des sujets de phrases (par exemple, "metro" dans l'avis d'exemple).

La fonction **detect_nouns**, quant à elle, a l'avantage de capturer tous les noms mentionnés dans l'avis, ce qui peut être utile pour avoir une vue d'ensemble des éléments mentionnés. Cependant, elle peut renvoyer des noms qui ne sont pas vraiment pertinents (par exemple, "hassle" dans l'avis d'exemple) et qui peuvent polluer les résultats.

Dans le contexte de notre projet, nous avons choisi d'utiliser la fonction detect_nouns pour extraire les informations de l'avis de voyage car nous souhaitons avoir une vue d'ensemble des éléments mentionnés dans l'avis, et non seulement les sujets de phrases. En effet, nous sommes intéressés par tous les noms qui sont mentionnés dans l'avis, qu'ils soient ou non des sujets de phrases, car ils peuvent nous fournir des informations précieuses sur les points forts et les points faibles de l'établissement mentionné dans l'avis. De plus, bien que la fonction detect_nouns puisse renvoyer des noms qui ne sont pas pertinents, nous pouvons utiliser un filtrage ultérieur pour éliminer ces noms et ne conserver que ceux qui sont réellement pertinents."

In [9]:
# on peut éliminer les commentaires trop long.
df['len']=df.subjects.apply(lambda x: len(x))
df=df.drop(df[df['len'] > 8].index)


## Aperçu des résultats


In [10]:
import pandas as pd
from collections import Counter
def count_words(words):
    # Use the Counter class to count the number of occurrences of each word
    count = Counter(words)
    return count

# Use the apply() method to apply the count_words() function to each row of the 'words' column
df['word_count'] = df['subjects'].apply(count_words)


In [11]:
result = []

# Use the extend() method to add each list to the result list
for row in df.subjects:
    result.extend(row)
count = Counter(result)
aa=pd.DataFrame(count.values(),count.keys())
aa.columns=['lenn']
aa=aa.sort_values(by='lenn')
aa=aa[::-1]
aa.loc[aa.lenn>50]

,lenn
staff,2773
room,2663
location,1795
rooms,1665
breakfast,986
...,...
elevator,56
downside,54
water,52
apartments,52


Il est intéressant de noter que certains mots ont un très grand nombre d'occurrences dans la base de données d'avis de voyage, tels que "staff" (2773 occurrences), "room" (2663 occurrences) et "location" (1795 occurrences). Cela peut être dû au fait que ces mots sont souvent utilisés par les voyageurs pour décrire les établissements hôteliers qu'ils ont visités, et sont donc très présents dans les avis.

Il est possible d'améliorer les résultats en utilisant une technique de filtrage des mots qui permet d'éliminer les mots les moins fréquents et de ne conserver que ceux qui sont les plus pertinents pour l'analyse. Par exemple, on pourrait définir un seuil de fréquence au-dessus duquel un mot est considéré comme pertinent et en dessous duquel il est ignoré. Cela permettrait de ne pas surcharger les résultats avec des mots peu fréquents qui n'apportent pas d'informations significatives."

## Partie analyse de sentiment

Dans un premier temps nous avons mis en place un modèle de reconnaissance des entités nommées (NER) qui nous permet de détecter les différents champs mentionnés dans les avis de voyage (par exemple, "restaurant", "personnel", etc.).

Dans cette deuxième partie, nous allons utiliser ces champs pour tenter de deviner le sentiment exprimé par l'utilisateur à leur sujet. Pour cela, nous allons utiliser des techniques d'analyse de sentiment qui nous permettront de classer chaque champ selon qu'il est positif, négatif ou neutre.

Cette étape de notre projet est cruciale car elle nous permettra de mieux comprendre ce que les utilisateurs aiment et n'aiment pas dans les établissements hôteliers qu'ils ont visités, et ainsi d'améliorer l'expérience de voyage des utilisateurs à l'avenir.

In [12]:
import spacy

nlp = spacy.load("en_core_web_sm")

def determine_sentiment(review, subjects):
    """Cette fonction prend en entrée un avis de voyage sous forme de chaîne de caractères et une liste de sujets, et renvoie un dictionnaire associant à chaque sujet sa valeur de sentiment.
    
    Pour déterminer le sentiment d'un sujet, la fonction utilise un modèle de traitement du langage naturel pour analyser chaque mot de l'avis et vérifier s'il s'agit d'un adjectif décrivant le sujet. Si l'adjectif est positif (par exemple, "fantastique", "excellent"), le sentiment du sujet est augmenté de 1, et s'il est négatif (par exemple, "déplorable", "horrible"), il est diminué de 1. Le sentiment d'un sujet est donc un nombre compris entre -1 et 1 qui reflète l'opinion de l'utilisateur à son sujet.
    
    :param review: L'avis de voyage à analyser, sous forme de chaîne de caractères.
    :type review: str
    :param subjects: La liste des sujets à analyser.
    
    """
    # Chargement du modèle de traitement du langage naturel
    doc = nlp(review)
    sentiments = {}
    # Pour chaque sujet
    for subject in subjects:
        # Initialisation du sentiment du sujet à 0
        sentiment = 0
        for token in doc:
            # Si le mot est le sujet en cours d'analyse
            if token.text == subject:
                # Pour chaque mot enfant du sujet (c'est-à-dire, les mots qui décrivent le sujet)
                for child in token.children:
                    # Si le mot enfant est un adjectif
                    if child.pos_ == "ADJ":
                        
                        # Si l'adjectif est positif
                        if child.text in ["amazing", "awesome", "brilliant", "excellent", "fantastic",
                                          "great", "incredible", "outstanding", "phenomenal", "superb",
                                          "terrific", "wonderful","efficient","helpful","efficient","outstading"]:
                            sentiment += 1
                        # Si l'adjectif est négatif
                        elif child.text in ["awful", "bad", "dreadful", "horrible", "terrible",
                                            "atrocious", "deplorable", "disgusting", "horrendous",
                                            "lamentable", "poor", "unacceptable", "unsatisfactory",]:
                            sentiment -= 1
        sentiments[subject] = sentiment
    return sentiments



In [13]:
rv

'fantastic position, close to the metro, from where all places are attainable. buy a five day metro ticket, saves the hassle of paying each time. clean rooms, helpful staff, modern facilities, wifi, breakfast is about €10 a-day. meats, cheese, cold breakfast as you say. no kettles in the rooms, for those who are not from europe. i would easy stay here again, location is everything, and a good price. valve for money, it has to be 10/10.'

In [14]:
determine_sentiment(rv, sbj2)

{'position': 1,
 'places': 0,
 'day': 0,
 'ticket': 0,
 'hassle': 0,
 'time': 0,
 'rooms': 0,
 'staff': 1,
 'facilities': 0,
 'wifi': 0,
 'breakfast': 0,
 'meats': 0,
 'cheese': 0,
 'kettles': 0,
 'location': 0,
 'price': 0,
 'money': 0}

"La fonction determine_sentiment a présenté de nombreuses limites dans sa capacité à déterminer les sentiments des utilisateurs de manière précise et fiable. L'une des principales raisons de cette limite est que la fonction ne peut pas détecter correctement les avis positifs et négatifs en raison des nuances présentes dans les avis.

Par exemple, un avis qui semble globalement positif peut contenir des éléments négatifs tels que des critiques ou des regrets, qui ne sont pas pris en compte par la fonction. De même, un avis qui semble globalement négatif peut contenir des éléments positifs tels que des louanges ou des remerciements, qui sont ignorés par la fonction.

Cela signifie que la fonction ne peut pas toujours donner un sentiment précis et véritable pour un sujet, ce qui limite grandement son utilité.

# ***********************Sentiment  analysis avec Vader***************************


Nous avons décidé de tester une méthode plus sophistiquée pour obtenir les sentiments des utilisateurs à partir de leurs avis de voyage. étant donné que nous n'avions pas suffisamment de données pour entraîner notre propre modèle de traitement du langage naturel, nous avons opté pour l'utilisation de l'analyseur de sentiments de VADER.

L'analyseur de sentiments de VADER est un outil pré-entraîné qui permet de détecter les sentiments positifs, négatifs et neutres dans du texte en utilisant une combinaison de lexiques et de règles de traitement du langage naturel. Nous espérons que cet outil nous permettra d'obtenir des résultats plus précis et fiables en matière de détermination des sentiments des utilisateurs.

In [15]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\moghazali\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Pour analyser les sentiments des utilisateurs sur chaque sujet détecté, nous devons d'abord diviser l'avis en phrases plus petites en utilisant des signes de ponctuation ou des mots-clés tels que "mais", "toutefois", etc. Cela nous permettra de mieux cibler chaque opinion exprimée par l'utilisateur et d'obtenir des résultats plus précis et fiables.

Ensuite, nous devrons analyser chaque phrase séparément pour déterminer le sentiment de l'utilisateur à l'égard du sujet en question. Cependant, nous ne devrons conserver que les phrases qui contiennent les entités que nous avons détectées dans la première partie du projet, car seules ces phrases sont pertinentes pour notre analyse.

En utilisant cette approche, nous espérons pouvoir obtenir une vue plus précise et détaillée des sentiments des utilisateurs envers chaque sujet détecté."

In [16]:

import numpy as np
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nlp = spacy.load('en_core_web_sm')


def Sentiment(text : str,words) :
    """
    param text : le review chaîne de caractère
    param words : liste des noms ou des sujets généré par les fonctins précédentes
    """
    # Conversion de la chaîne de caractères en minuscule
    text = text.lower()
    # Division de la chaîne de caractères en phrases plus petites en utilisant des signes de ponctuation et des mots-clés
    text = re.split(r'[\.!?;,]| but | whereas | yet | although | though | still | furthermore | moreover | even though | on the other hand | in spite of ', text)
    
    # Initialisation d'une liste vide pour stocker les tokens
    list_token = []
    model = SentimentIntensityAnalyzer()
    # Initialisation d'une liste vide pour stocker les tuples (entité, sentiment)
    aa=[]
    #Parcours de chaque phrase
    for phrase in text :
        # Vérification si l'une des entités est présente dans la phrase
        if any(word in phrase for word in words):
            # Détermination du sentiment de l'utilisateur à l'égard de l'entité grâce à l'analyseur de sentiments de VADER
            sentiment = model.polarity_scores(phrase).get("compound")
            # Recherche de l'entité dans la phrase
            for word in words:
                if word in phrase:
                    # # Entité trouvée dans la phrase, on la stocke dans la variable found_word
                    found_word = word
                    break
            # Ajout du tuple (entité, sentiment) à la liste
            aa.append((found_word,sentiment))
    # Renvoi de la liste des tuples (entité, sentiment)
    return aa 
 


In [17]:
df['sentiment']=df.apply(lambda row: Sentiment( row['review'], row['nouns']), axis=1)

In [18]:
df

,index,title,review,rate,trip_type,date,subjects,nouns,len,word_count,sentiment
0,0,small room but okey,"this hotel has the smallest room, and we have ...",4,traveled as a couple,2022-11-01,"[check, staffs, breakfast, location, metro]","[hotel, room, brand, hotels, globe, staffs, br...",5,"{'check': 1, 'staffs': 1, 'breakfast': 1, 'loc...","[(hotel, 0.0), (hotel, 0.0), (staffs, 0.0), (b..."
1,1,"great location, great staff",great location and simple but clean ammenities...,5,traveled on business,2022-10-01,[staff],"[location, ammenities, value, money, stars, st...",1,{'staff': 1},"[(location, 0.6249), (ammenities, 0.4019), (va..."
2,2,dont stay in this hotel unless you would like ...,don't stay in this hotel unless you would like...,1,NaN,2022-10-01,"[reservation, staff, room, staff]","[hotel, cancellation, email, day, check, room,...",4,"{'reservation': 1, 'staff': 2, 'room': 1}","[(hotel, 0.3612), (room, 0.0), (flight, 0.0), ..."
3,3,waste of money,not worth it. better rooms available for this ...,1,NaN,2022-09-01,"[rooms, location, 20min, staff]","[rooms, price, rooms, bathrooms, location, wal...",4,"{'rooms': 1, 'location': 1, '20min': 1, 'staff...","[(rooms, 0.4404), (rooms, 0.0), (location, 0.4..."
4,4,pre-autorisation money has not been refunded,ibis paris tour eiffel cambronne charged my de...,3,traveled as a couple,2022-10-01,"[cambronne, money]","[eiffel, cambronne, debit, account, eur, pre, ...",2,"{'cambronne': 1, 'money': 1}","[(eiffel, -0.2023), (eur, 0.0), (pre, -0.2023)..."
...,...,...,...,...,...,...,...,...,...,...,...
8564,907,just papering over the cracks!,just got back from 3 nights in paris\none nigh...,3,traveled on business,2010-01-01,"[location, areas, lounge, rooms, staff]","[nights, night, star, nights, hotel, location,...",5,"{'location': 1, 'areas': 1, 'lounge': 1, 'room...","[(nights, 0.0), (nights, 0.2006), (b, 0.9186),..."
8565,908,perfect,our stay was perfect. the location was amazing...,5,traveled as a couple,2008-08-01,"[stay, location, room]","[stay, location, stones, room, view]",3,"{'stay': 1, 'location': 1, 'room': 1}","[(stay, 0.5719), (location, 0.5859), (stones, ..."
8566,909,very nice hotel,we decided on this hotel in paris for location...,4,traveled with family,2009-05-01,"[employees, service, bathrooms, fi, signal, br...","[hotel, location, view, count, location, rate,...",7,"{'employees': 1, 'service': 1, 'bathrooms': 1,...","[(hotel, 0.0), (count, 0.3724), (location, 0.0..."
8567,910,ideally situated,very small hotel but ideally situated with bre...,4,traveled as a couple,2008-07-01,"[rooms, hallways, room, staff, breakfast, croi...","[hotel, views, river, room, boutique, hotels, ...",7,"{'rooms': 1, 'hallways': 1, 'room': 1, 'staff'...","[(hotel, 0.0), (views, 0.7003), (hotel, 0.3612..."


exemple

In [19]:
df.sentiment.iloc[94]

[('rooms', 0.296),
 ('hotel', 0.7269),
 ('areas', 0.0),
 ('areas', 0.34),
 ('area', 0.2023),
 ('downside', -0.25),
 ('floor', -0.34),
 ('lift', 0.0),
 ('renovation', -0.2944)]

Cette fonction peut être utile pour obtenir une vue d'ensemble des sentiments de l'utilisateur et comprendre comment il a perçu différents aspects de l'hôtel. En utilisant cette fonction, nous pouvons avoir une idée de la manière dont les différentes entités sont perçues par les utilisateurs et identifier les points forts et les points faibles de l'hôtel. Cela peut nous aider à améliorer l'expérience des clients et à offrir un service de meilleure qualité.

Cependant, il convient de noter que cette approche n'est pas parfaite et peut parfois ne pas être en mesure de détecter les nuances ou l'ironie présentes dans l'avis, en particulier si le sujet est complexe ou ambigu. Pour améliorer les résultats de cette fonction, il serait utile d'utiliser un modèle de transformer BERT (Bidirectional Encoder Representations from Transformers) pour l'analyse de sentiments et la détection des entités. BERT est un modèle de traitement du langage naturel performant qui a été entraîné sur de vastes quantités de données et est connu pour ses performances exceptionnelles dans la reconnaissance des entités et la détermination de leur relation avec le reste du texte. En utilisant BERT, nous pourrions obtenir une meilleure précision dans la détection des sentiments de l'utilisateur et des entités dans les avis. Cela nous permettrait d'améliorer l'expérience des clients et d'offrir un service de meilleure qualité.